# [IAPR 2019:][iapr2019] Special project

**Group members:**
    1- Emma Nilsson,
    2- Anna Fredriksson Häägg,
    3- Herman Kristian Dieset

**Due date:** 30.05.2019

[iapr2019]: https://github.com/LTS5/iapr-2019


## Description
Please find the description of this special project via [this link].

[this link]: https://github.com/LTS5/iapr-2019/blob/master/project/special_project_description.pdf

# Imports

In [1]:
import tarfile
import os
import scipy.io
import skimage.io
import skimage.color
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import xml.etree.ElementTree as ET
import cv2
import numpy as np
from sklearn import preprocessing
from sklearn import svm
from skimage.color import rgb2gray
import skimage.morphology as mp
from skimage import measure
import tensorflow as tf

#import sys
from IPython.display import clear_output

In [2]:
#data_base_path = os.path.join(os.pardir, 'data') # only works for Herman
data_base_path = ('data')
data_folder = 'project-data'
tar_path = os.path.join(data_base_path, data_folder + '.tar.gz')
#with tarfile.open(tar_path, mode='r:gz') as tar:
#    tar.extractall(path=data_base_path)

We create functions for loading images and annotations and make sure that each index match.

In [3]:
def load_images():

    train_dir = os.path.join(data_base_path, data_folder +'/images/train/')
    dir_train_list = sorted(os.listdir(train_dir))
    train = [skimage.io.imread(os.path.join(train_dir, file)) for file in dir_train_list]

    
    test_dir = os.path.join(data_base_path, data_folder +'/images/test/')
    dir_test_list = sorted(os.listdir(test_dir))
    test = [skimage.io.imread(os.path.join(test_dir, file)) for file in dir_test_list]
    
    val_dir = os.path.join(data_base_path, data_folder +'/images/validation/')
    dir_val_list = sorted(os.listdir(val_dir))
    validation = [skimage.io.imread(os.path.join(val_dir, file)) for file in dir_val_list]
    
    return train, test, validation

In [4]:
def parse_xml_file(filename):
    """ Parse a PASCAL VOC xml file """
    tree = ET.parse(filename)
    objects = []
    for obj in tree.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(float(bbox.find('xmin').text)),
                              int(float(bbox.find('ymin').text)),
                              int(float(bbox.find('xmax').text))-int(float(bbox.find('xmin').text)),
                              int(float(bbox.find('ymax').text))-int(float(bbox.find('ymin').text))]
        objects.append(obj_struct)

    return objects

In [6]:
def load_annotations():
    dir_train = os.path.join(data_base_path, data_folder + '/annotations/train/')
    dir_train_list = sorted(os.listdir(dir_train))
    train_files = [f for f in dir_train_list if os.path.isfile(os.path.join(dir_train, f))]
    train_annotations = [parse_xml_file(os.path.join(dir_train, file)) for file in train_files]
    
    
    dir_test = os.path.join(data_base_path, data_folder + '/annotations/test/')
    dir_test_list = sorted(os.listdir(dir_test))
    test_files = [f for f in dir_test_list if os.path.isfile(os.path.join(dir_test, f))]
    test_annotations = [parse_xml_file(os.path.join(dir_test, file)) for file in test_files]
    
    dir_val = os.path.join(data_base_path, data_folder + '/annotations/validation/')
    dir_val_list = sorted(os.listdir(dir_val))
    val_files = [f for f in dir_val_list if os.path.isfile(os.path.join(dir_val, f))]
    validation_annotations = [parse_xml_file(os.path.join(dir_val, file)) for file in val_files]
    
    return train_annotations, test_annotations, validation_annotations

In [7]:
train, test, validation = load_images()
train_annotations, test_annotations, validation_annotations = load_annotations()

# Preprocessing 


In [8]:
def remove_yellow(img):
    """
    Remove yellow tones from the image.
    """
    out = img.copy()
    
    # Get different channels
    red = out[:,:,0]
    green = out[:,:,1]
    blue = out[:,:,2]
    
    # Define yellow and set all these parts to white in the image
    is_yellow =  (red > 100) & (red < 250) & (green > 100) & (green < 250) & (blue > 0) & (blue < 200)
    red[is_yellow] = 255
    green[is_yellow] = 255
    blue[is_yellow] = 255
    
    out[:,:,0] = red
    out[:,:,1] = green
    out[:,:,2] = blue
    
    return out

In [9]:
def threshold(image, th1, th2):
    """Threshold an image between two thresholds."""
    th_img = image.copy()
    th_img[th_img<th1] = 0
    th_img[th_img>th2] = 0
    th_img[(th_img>=th1) & (th_img<=th2)] = 255
    return th_img

In [10]:
def erode(img, nb):
    """Apply erosion to image."""
    out = img.copy()
    for i in range(nb):
        out = mp.erosion(mp.erosion(skimage.img_as_ubyte(img)))
    return out

In [375]:
def preprocess_image(img):
    """Apply entire pre-processing pipeline to an image."""
    out = img.copy()
    out = remove_yellow(out)
    img_g = rgb2gray(out)
    img_th = threshold(skimage.img_as_ubyte(img_g), 1, 100)
    img_er = erode(img_th,1)
    return img_er

In [12]:
def preprocess_images(train, test, validation):
    """Pre-process all images."""
    train_out = train.copy()
    test_out = test.copy()
    val_out = validation.copy()
    
    for i in range(len(train_out)):
        train_out[i] = preprocess_image(train_out[i])
        
    for i in range(len(test_out)):
        test_out[i] = preprocess_image(test_out[i])
        
    for i in range(len(val_out)):
        val_out[i] = preprocess_image(val_out[i])
         
    return train_out, test_out, val_out


In [13]:
train_pro, test_pro, validation_pro = preprocess_images(train, test, validation)

/anaconda3/envs/ada/lib/python3.6/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [14]:
print("Nb train images     : {}".format(len(train_pro)))
print("Nb test images      : {}".format(len(test_pro)))
print("Nb validation images: {}".format(len(validation_pro)))

Nb train images     : 800
Nb test images      : 50
Nb validation images: 2


## Help functions for validation

In [15]:
def precision(tp, fp):
    return tp / (tp + fp)

def recall(tp, fn):
    return tp / (tp + fn)

def f1_score(tp, fp, fn):
    if (tp == 0):
        return 0
    return 2 * precision(tp, fp) * recall(tp, fn) / (precision(tp, fp) + recall(tp, fn))

def iou_bbox(predict, true):
    """
    Calculate the IoU for two bounding boxes.
    Based on:
    https://stackoverflow.com/questions/25349178/calculating-percentage-of-bounding-box-overlap-for-image-detector-evaluation
    """
    if type(predict[0]) == list or type(true[0]) == list :
        print((predict))
        print(true)
        #return 0
    # Determine the coordinates of the intersection rectangle
    x_left   = max(predict[0], true[0])
    y_bottom = max(predict[1], true[1])
    x_right  = min(predict[0] + predict[2], true[0] + true[2])
    y_top    = min(predict[1] + predict[3], true[1] + true[3])

    # If they do not overlap, return 0
    if x_right < x_left or y_bottom > y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_top - y_bottom)

    # Compute area for both bounding boxes
    bb1_area = predict[2] * predict[3]
    bb2_area = true[2] * true[3]

    # Compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the intersection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)

    return iou

## Validation function

We define a function which validates how a method works by setting IoU=0.3 and calculating precision, recall and F1-score.

In [317]:
def validate_method(method, test_between = None, printVals = False, test_imgs = test_pro):

    def update_progress(progress):
        """Prints progress of the used validation method."""
        bar_length = 30
        if isinstance(progress, int):
            progress = float(progress)
        if not isinstance(progress, float):
            progress = 0
        if progress < 0:
            progress = 0
        if progress >= 1:
            progress = 1
            
        block = int(round(bar_length * progress))
        clear_output(wait = True)
        text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
        print(text)

    tp = 0
    fp = 0
    fn = 0
    
    # Setup progress bar
    limit = len(test_imgs[test_between[0]:test_between[1]] if test_between != None else test_imgs)

    for i in range(limit):
        update_progress(i/limit)
        # Predict varroas
        varroas_pred = method(test_imgs[i])

        # Get true bounding boxes
        varroas_true = []
        for bbox in test_annotations[i]:
               varroas_true.append(bbox['bbox'])

        # Get true and false positives
        positives = []

        for pb in varroas_pred:
            true_pos = False

            for tb in varroas_true:
                if(iou_bbox(pb, tb) > 0.3):
                    true_pos = True
                    break;
            positives.append(true_pos)

        tp += positives.count(True)
        fp += positives.count(False)

        # Get the false negatives
        negatives = []

        for tb in varroas_true:
            fn = False

            for pb in varroas_pred:
                if(iou_bbox(pb, tb) > 0.3):
                    fn = True
                    break;
            negatives.append(fn)

        # Count all times no intersection was found
        fn += negatives.count(False)

    update_progress(1)
    
    if printVals :
        print('The values for the given test images')
        print(tp)
        print(fn)
        print(fp)

    
    print("\n----------- SCORES -----------")
    print("Precision: {}".format(precision(tp,fp)))
    print("Recall   : {}".format(recall(tp,fn)))
    print("F1-score : {}".format(f1_score(tp,fp,fn)))
    print("------------------------------")

## Part 1: Finding varroas by segmentation
Add your implementation for ''**detect_by_segmentation**'' function. Please make sure the input and output follows the mentioned format.

In [19]:
def find_contours(img):
    contours = measure.find_contours(img, 0)
    lengths = []
    for cnt in contours:
        lengths.append(len(cnt))
    return contours, lengths

In [20]:
def contour_threshold_length(contours, th_low = 0, th_high = np.Inf) :
    # Filters list of contours, keeps those within threshold limits
    out = []
    for n, contour in enumerate(contours):
        if len(contour) > th_low and len(contour) < th_high :
            out.append(contour)
    return out, len(out)

In [21]:
def contours2boxlist(contours):
    boxes = []
    for contour in contours :
        x_min = int(np.min(contour[:,1]))
        x_max = int(np.max(contour[:,1]))
        y_min = int(np.min(contour[:,0]))
        y_max = int(np.max(contour[:,0]))
        
        boxes.append([x_min, y_min, x_max - x_min, y_max - y_min])
    
    return boxes # [[x_1, y_1, w_1, h_2]

In [22]:
def detect_by_segmentation(img):
    '''
    Input: One single image
    Output: A numpy array containing coordonates of all detected varroas, with the following format: 
            [[x_1, y_1, w_1, h_2], [x_2, y_2, w_1, h_2], ..., [x_n, y_n, w_n, h_n]] 
            where ''n'' is the number of detected varroas.
    '''
    #img = preprocess_image(img)
    contours,_ = find_contours(img)
    contours,_ = contour_threshold_length(contours,40,80)
    boxlist = contours2boxlist(contours)
    
    return boxlist

Add your implementation. Report the Precision, Recall and F1-score, by using all 50 images of the test-set, and considering 0.3 as the IoU threshold.

In [39]:
validate_method(detect_by_segmentation)

Progress: [##############################] 100.0%

----------- SCORES -----------
Precision: 0.04203670811130847
Recall   : 0.9861111111111112
F1-score : 0.08063600227143669
------------------------------


## Part 2: Implement your first detector

Write your function(s) for the second part. Feel free to change the name of the function and add your additional functions, but please make sure their input and output follows the mentioned format.

We define a sliding window function to be able to slide over the image and detect objects at different parts of the image.

In [17]:
def sliding_window(image, stepSize, windowSize):
    # slide a window across the image
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            # yield the current window
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

We need to create our training features for the classifier. This is done by first finding the contours for all training annotations and then using them to compute the first 6 fourier descriptors of all contours. These will be our features.

In [349]:
def get_contours(images, annotations):
    '''
    Input: Training images and training annotations
    Output: Max length of the contours and an array with the contours of all training varroas
    '''
    max_len = 0
    all_contours = []
    
    for i in range(len(images)):
        for anno in annotations[i]:
            bbox = anno['bbox']
            img_box = images[i][bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]].copy()
            im_bw = img_box

            # Find contours using opencv
            _, contours, _ = cv2.findContours(im_bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

            # Reshape to easily get x and y
            if (len(contours) != 0):
                contours_np = contours[0][:, 0, :]
                all_contours.append(contours_np)
                
                # See if max length of contours has changed
                if (max_len < contours_np.shape[0]):
                    max_len = contours_np.shape[0]

    return max_len, np.array(all_contours)

All our contours have the same length, so we need to pad them to calculate the fourier descriptors. Since we pad with 0, this might cause some errors to our data but we figure it works well enough.

In [350]:
def fourier_descriptor(contours, length):
    '''
    Input: one contour, max length of all contours
    Output: The 10 first fourrier descriptors of the contour
    '''
    
    # Pad contour to max contour length
    for i in range(length - len(contours)):
        contours = np.concatenate([contours, np.array([[0, 0]])])
    
    # Create imag nums and descriptors
    u_k = contours[:, 0] + 1j*contours[:, 1]
    
    fr = np.fft.fft(u_k)
    
    # Only return the 2nd and 3rd
    return [fr[1], fr[2], fr[3], fr[4], fr[5], fr[6]]

Create and normalize training features.

In [351]:
max_len, contours = get_contours(train_pro, train_annotations)
features = np.abs([(fourier_descriptor(c, max_len)) for c in contours])

In [352]:
features = np.c_[ features, np.zeros(features.shape[0]) ]  
scaler = preprocessing.StandardScaler().fit(features[:, 0:6])
features_s = scaler.transform(features[:, 0:6])

Train a classifier.

In [353]:
clf_svm_one = svm.OneClassSVM(nu=0.9)
clf_svm_one.fit(features_s)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto_deprecated',
      kernel='rbf', max_iter=-1, nu=0.9, random_state=None, shrinking=True,
      tol=0.001, verbose=False)

### Test the detection

To not miss any varroas the sliding window will take quite small steps (6-8px). This means that one varroa might be found several times. A merge_box function is implemented to merge overlapping bounding boxes before returning them.

In [354]:
def merge_boxes(bboxes_in):
    bboxes = bboxes_in.copy()
    for i in range(len(bboxes)):
        for j in range(i+1, len(bboxes)):
            if(iou_bbox(bboxes[i], bboxes[j]) > 0.3):

                # Determine the coordinates of the intersection rectangle
                x_left = min(bboxes[i][0], bboxes[j][0])
                y_top = min(bboxes[i][1], bboxes[j][1])
                x_right = max(bboxes[i][0] + bboxes[i][2], bboxes[j][0] + bboxes[j][2])
                y_bottom = max(bboxes[i][1] + bboxes[i][3], bboxes[j][1] + bboxes[j][3])
                
                bboxes[j] = [x_left, y_top, x_right - x_left, y_bottom - y_top]
                del bboxes[i]
                break
    return bboxes

In [355]:
def detect_by_method_1(image):
    '''
    Input: One single image
    Output: A numpy array containing coordonates of all detected varroas, with the following format: 
            [[x_1, y_1, w_1, h_2], [x_2, y_2, w_1, h_2], ..., [x_n, y_n, w_n, h_n]] 
            where ''n'' is the number of detected varroas.
    '''
    
    window_shape = (45, 45)
    padding = max_len
    varroas = []
    for window in sliding_window(image, 6, window_shape):
        im_bw = window[2]

        # Find contours using opencv
        _, contours, _ = cv2.findContours(im_bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)


        # Reshape to easily get x and y
        if (len(contours) > 0):
            contours_np = contours[0][:, 0, :]
            descriptor = np.abs(fourier_descriptor(contours_np, padding))
            test_ = scaler.transform(np.array([descriptor[:]]))

            if (prob > 0):
                varroas.append([window[0], window[1], window_shape[0], window_shape[1]])

    return merge_boxes(varroas)

Validation of result by repporting the Precision, Recall and F1-score, by using all 50 images of the test-set, and considering 0.3 as the IoU threshold.

In [356]:
validate_method(detect_by_method_1)

Progress: [##############################] 100.0%

----------- SCORES -----------
Precision: 0.10279187817258884
Recall   : 0.9529411764705882
F1-score : 0.18556701030927833
------------------------------


From our high recall but really low precision score, we can conclude that we catch most varroas, but also a lot of objects which are not varroas. Improved preprocessing for the images might have helped us elevate the precision, but we think deep learning will be the best solution to the problem.

## Part 3: Using MLP and CNNs

Add your implementation for the thrid part. Feel free to add your desirable functions, but please make sure you have proper functions for the final detection, where their input and output follows the same format as the previous parts.

Firstly we create training data by using the parts of images we know contain varroas.

In [329]:
def get_varroa_images(images, annotations, width, height, channels=3):
    '''
    Input: Training images and training annotations
    Output: An array with all varroa images
    '''
    varroas = []
    for i in range(len(images)):
        for anno in annotations[i]:
            bbox = anno['bbox']
            img_box = images[i][bbox[1]:bbox[1]+height, bbox[0]:bbox[0]+width].copy()

            if (channels != 3):
                img_box = np.expand_dims(img_box, axis=2)

            if (img_box.shape == (height, width, channels)):
                varroas.append(img_box)

    return varroas

To train the classifier we need a negative class that represent images without varroas. To create a dataset for this we find the images without varroas, slide over them, and save some windows to our negative class. 

In [330]:
def create_second_class_images(images, annotations, width, height, channels=3):
    # Find all images without varroas
    empty_anno_index = []
    for i in range(len(annotations)):
        if len(annotations[i]) == 0:
            empty_anno_index.append(i)

    # Store the first 5 images (change later)
    #print(len(empty_anno_index))
    #empty_anno_index = empty_anno_index[:100]
    empty_pics = [images[i] for i in empty_anno_index]
    
    no_varroas = []
    
    # Slide over images and append all windows to the second class data array
    for image in empty_pics:
        for window in sliding_window(image, 300, (height, width)):
            no_varroa_img = window[2]
            if (channels != 3):
                no_varroa_img = np.expand_dims(no_varroa_img, axis=2)
            if (no_varroa_img.shape == (height, width, channels)):
                no_varroas.append(no_varroa_img)

    return no_varroas

Create train features and save them.

In [331]:
window_shape = (45, 45)
varroas_pos = np.stack(get_varroa_images(train, train_annotations, window_shape[0], window_shape[1]), axis=0)
varroas_neg = np.stack(create_second_class_images(train, train_annotations, window_shape[0], window_shape[1]), axis=0)
train_features = np.vstack((varroas_pos, varroas_neg))

In [332]:
# Check features shape
train_features.shape

(17450, 45, 45, 3)

Create train labels.

In [333]:
label_pos = np.ones((varroas_pos.shape[0]))
label_neg = np.zeros((varroas_neg.shape[0]))
train_labels = np.concatenate((label_pos, label_neg))

In [334]:
train_features_float = train_features.astype(np.float32)

### Keras imports, helpers etc

In [358]:
from tensorflow import keras 
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Flatten
from keras.models import Model, Sequential
from keras import optimizers

We randomize the data we use when training. Since we have already created the features for the train dataset but not for validation, we split the data so we can use part of the train dataset as validation set. To be able to use more data, this could be improved by insted using the validation data set.

In [336]:
def split_data(x, y, ratio=0.8, seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    #np.random.seed(seed)
    
    # generate random indices
    num_row = len(y)
    indices = np.random.permutation(num_row)
    index_split = int(np.floor(ratio * num_row))
    index_tr = indices[: index_split]
    index_te = indices[index_split:]
    
    # create split
    x_tr = x[index_tr]
    x_te = x[index_te]
    y_tr = y[index_tr]
    y_te = y[index_te]
    return x_tr, x_te, y_tr, y_te

In [369]:
# Reshaping data for MLP
train_linear = train_features_float.reshape((-1, 45*45*3)).astype(np.float32)

In [370]:
x_train, x_test, y_train, y_test = split_data(train_linear/255, train_labels)
y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)

### MLP

We decided to have a MLP with five hidden layers as we have quite many features. We decided to use Adam as an optimizer because it worked slightly better than SGD.

In [371]:
model_mlp = Sequential()

model_mlp.add(Dense(2**10, activation='relu', input_dim=x_train.shape[1]))
model_mlp.add(Dense(2**8, activation='relu'))
model_mlp.add(Dense(2**6, activation='relu'))
model_mlp.add(Dense(2**4, activation='relu'))
model_mlp.add(Dense(2**3, activation='relu'))

model_mlp.add(Dense(2, activation='softmax'))

adam = optimizers.Adam()
model_mlp.compile(
    loss='binary_crossentropy',
    optimizer=adam,
    metrics=['accuracy']
)

In [372]:
model_mlp.fit(
        x_train,
        y_train,
        batch_size=100,
        epochs=5,
        validation_data=(x_test, y_test)
    )

Train on 13960 samples, validate on 3490 samples
Epoch 1/5
13960/13960 [==============================] - 12s 845us/step - loss: 0.4056 - acc: 0.7871 - val_loss: 0.1397 - val_acc: 0.9613
Epoch 2/5
13960/13960 [==============================] - 11s 783us/step - loss: 0.1257 - acc: 0.9638 - val_loss: 0.1106 - val_acc: 0.9693
Epoch 3/5
13960/13960 [==============================] - 11s 823us/step - loss: 0.1085 - acc: 0.9695 - val_loss: 0.0902 - val_acc: 0.9762
Epoch 4/5
13960/13960 [==============================] - 11s 820us/step - loss: 0.0908 - acc: 0.9739 - val_loss: 0.1144 - val_acc: 0.9711
Epoch 5/5
13960/13960 [==============================] - 12s 852us/step - loss: 0.0884 - acc: 0.9746 - val_loss: 0.0874 - val_acc: 0.9768


### Predictions

In [373]:
def detect_by_mlp(image):
    '''
    Input: One single image
    Output: A numpy array containing coordonates of all detected varroas, with the following format: 
            [[x_1, y_1, w_1, h_2], [x_2, y_2, w_1, h_2], ..., [x_n, y_n, w_n, h_n]] 
            where ''n'' is the number of detected varroas.
    '''
    
    window_shape = (45, 45)
    varroas = []
    step = 20
    
    for i,window in enumerate(sliding_window((image)/255, step, window_shape)):
        im_bw = window[2]
        
        # Only check when the window is the shape accepted by the MLP
        if (im_bw.shape == (window_shape[0], window_shape[1], 3)):
            # Get image and predict probability
            im_bw = np.reshape(im_bw, [1, 45*45 * 3])
            prob = model_mlp.predict(im_bw)

            # Append if there is a high probability that the window contains a varroa
            if (prob[0][1] > 0.95):
                varroas.append([window[0], window[1], window_shape[0], window_shape[1]])

    return merge_boxes_improved(varroas)

In [374]:
# Reshaping data for MLP
validate_method(detect_by_mlp, test_between=(0, 50), test_imgs=test)

Progress: [##############################] 100.0%

----------- SCORES -----------
Precision: 0.13131313131313133
Recall   : 0.9512195121951219
F1-score : 0.23076923076923078
------------------------------


### CNN

We define a CNN with 3 convolutional layers. All layers use the kernel size 5x5, and max_pooling has the pool_size=(2, 2). We use one linear layer after the last max pooling layer before we calculate the output.

In [341]:
# Create input layer
input_ = Input(shape=(45, 45, 3), dtype='float32')

# Create convolution and max pooling layers
conv1 = Conv2D(filters=16, kernel_size=5, padding='valid', activation='relu', strides=1)(input_)
max1 = MaxPooling2D(pool_size=(2,2))(conv1)
conv2 = Conv2D(filters=32, kernel_size=5, padding='valid', activation='relu', strides=1)(max1)
max2 = MaxPooling2D(pool_size=(2,2))(conv2)
conv3 = Conv2D(filters=64, kernel_size=5, padding='valid', activation='relu', strides=1)(max2)
max3 = MaxPooling2D(pool_size=(2,2))(conv3)

# Flatten
flatten = Flatten()(max3)

# Dropout, activation
linear = Dense(256, activation='relu')(flatten)
output = Dense(2, activation='softmax')(linear)

# Create and compile model
# Both sgd and adam has been tested as optimizers, adam gave the best results
adam = optimizers.Adam()
model_cnn = Model(inputs=[input_], outputs=[output])
model_cnn.compile(
    loss='binary_crossentropy',
    optimizer=adam,
    metrics=['accuracy']
)

We normalize the data by dividing it with 255 before sending it to be splitted in training and validation sets.

In [342]:
x_train, x_test, y_train, y_test = split_data(train_features_float/255, train_labels)
y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)

Since we quickly achieve high validation accuracy, the model is trained with 5 epochs. The batch size is set to 100.

In [343]:
model_cnn.fit(
        x_train,
        y_train,
        batch_size=100,
        epochs=5,
        validation_data=(x_test, y_test)
    )

Train on 13960 samples, validate on 3490 samples
Epoch 1/5
13960/13960 [==============================] - 20s 1ms/step - loss: 0.1309 - acc: 0.9575 - val_loss: 0.0546 - val_acc: 0.9834
Epoch 2/5
13960/13960 [==============================] - 21s 1ms/step - loss: 0.0590 - acc: 0.9822 - val_loss: 0.0425 - val_acc: 0.9848
Epoch 3/5
13960/13960 [==============================] - 21s 2ms/step - loss: 0.0525 - acc: 0.9830 - val_loss: 0.0411 - val_acc: 0.9857
Epoch 4/5
13960/13960 [==============================] - 21s 2ms/step - loss: 0.0480 - acc: 0.9840 - val_loss: 0.0360 - val_acc: 0.9894
Epoch 5/5
13960/13960 [==============================] - 21s 1ms/step - loss: 0.0429 - acc: 0.9862 - val_loss: 0.0359 - val_acc: 0.9911


### Predictions

In [344]:
def merge_boxes_cv(bboxes_in):
    rects, weights = cv2.groupRectangles(bboxes_in, groupThreshold=1, eps=1)
    return rects

In [345]:
def detect_by_cnn(image):
    '''
    Input: One single image
    Output: A numpy array containing coordonates of all detected varroas, with the following format: 
            [[x_1, y_1, w_1, h_2], [x_2, y_2, w_1, h_2], ..., [x_n, y_n, w_n, h_n]] 
            where ''n'' is the number of detected varroas.
    '''
    
    window_shape = (45, 45)
    varroas = []
    for window in sliding_window((image)/255, 15, window_shape):
        im_bw = window[2]

        if (im_bw.shape == (window_shape[0], window_shape[1], 3)):
            im_bw = np.reshape(im_bw, [1, 45, 45, 3])
            prob = model_cnn.predict(im_bw)
        
            if (prob[0][1] > 0.95):
                varroas.append([window[0], window[1], window_shape[0], window_shape[1]])

    return merge_boxes_cv(varroas)

In [348]:
validate_method(detect_by_cnn, test_imgs=test)

Progress: [##############################] 100.0%

----------- SCORES -----------
Precision: 0.21448087431693988
Recall   : 0.98125
F1-score : 0.3520179372197309
------------------------------


## Challenge

You can generate a json submission file by using the function ''**generate_pred_json**''. This prediction file can be uploaded online for evaluation (Please refer to section 3 of the project description for more details).

In [6]:
import numpy as np
import json

def generate_pred_json(data, tag='baseline'):
    '''
    Input
    - data: Is a dictionary d, such that:
          d = { 
              "ID_1": [], 
              "ID_2": [[x_21, y_21, w_21, h_21], [x_22, y_22, w_22, h_22]], 
              ... 
              "ID_i": [[x_i1, y_i1, w_i1, h_i1], ..., [x_iJ, y_iJ, w_iJ, h_iJ]],
              ... 
              "ID_N": [[x_N1, y_N1, w_N1, h_N1]],
          }
          where ID is the string id of the image (e.i. 5a05e86fa07d56baef59b1cb_32.00px_1) and the value the Kx4 
          array of intergers for the K predicted bounding boxes (e.g. [[170, 120, 15, 15]])
    - tag: (optional) string that will be added to the name of the json file.
    Output
      Create a json file, "prediction_[tag].json", conatining the prediction to EvalAI format.
    '''
    unvalid_key = []
    _data = data.copy()
    for key, value in _data.items():
        try:
            # Try to convert to numpy array and cast as closest int
            print(key)
            v = np.around(np.array(value)).astype(int)
            # Check is it is a 2d array with 4 columns (x,y,w,h)
            if v.ndim != 2 or v.shape[1] != 4:
                unvalid_key.append(key)
            # Id must be a string
            if not isinstance(key, str):
                unvalid_key.append(key)
            _data[key] = v.tolist()
        # Deal with not consistant array size and empty predictions
        except (ValueError, TypeError):
            unvalid_key.append(key)
    # Remove unvalid key from dictionnary
    for key in unvalid_key: del _data[key]
    
    with open('prediction_{}.json'.format(tag), 'w') as outfile:
        json.dump(_data, outfile)

In [7]:
#Your code

## Appendix: plotting functions

In [14]:
def plot_image(img):
    fig, ax = plt.subplots(1,1, figsize=(6,6))
    size = img.shape
    if len(size) == 3:
        ax.imshow(img)
        ax.set_title('({} px, {} px, depth {})'.format(size[0], size[1], size[2]))
    else:
        ax.imshow(img, cmap='gray')
        ax.set_title('({} px, {} px), single channel'.format(size[0], size[1]))
    plt.show()

In [24]:
def plot_with_annotation(img_nb, annotations, img = None):
    if annotations == [] : 
        print("No varroa here!")
        return None
    fig,ax = plt.subplots(1,1,figsize=(6,6))
    for anno in annotations:
        rect = patches.Rectangle((anno['bbox'][0], 
                                  anno['bbox'][1]), 
                                  anno['bbox'][2],
                                  anno['bbox'][3],
            linewidth=1,edgecolor='r',facecolor='none')
        ax.add_patch(rect)
    if img is not None: # img from argument
        size = img.shape
        if len(size) == 3:
            ax.imshow(img)
            ax.set_title('img nb {}, no varroae {}, ({} px, {} px, depth {})'.format(
                img_nb,len(annotations_xmls[img_nb]), size[0], size[1], size[2]))
        else:
            ax.imshow(img, cmap='gray')
            ax.set_title('img nb {}, no varroae {}, ({} px, {} px), single channel'.format(
                img_nb,len(annotations_xmls[img_nb]), size[0], size[1]))
    else: # image from collection
        size = img_collection[img_nb].shape
        if len(size) == 3:
            ax.imshow(img_collection[img_nb])
            ax.set_title('img nb {}, no varroae {}, ({} px, {} px, depth {})'.format(
                img_nb,len(annotations_xmls[img_nb]), size[0], size[1], size[2]))
        else:
            ax.imshow(img_collection[img_nb], cmap='gray')
            ax.set_title('img nb {}, no varroae {}, ({} px, {} px), single channel'.format(
                img_nb,len(annotations_xmls[img_nb]), size[0], size[1]))
    plt.show()

In [16]:
def plot_contours(img, th_low = 0, th_high = np.Inf):
    # Find contours
    contours,l = find_contours(img)
    plt.subplots(1,1,figsize=(6,6))

    plt.gca().invert_yaxis()
    
    contours,l = contour_threshold_length(contours, th_low, th_high)
    
    for contour in contours:
        plt.plot(contour[:, 1], contour[:, 0], linewidth=2)
    
    plt.title("{} contours between threholds {} and {}".format(l,
                                                              th_low,
                                                              th_high))
    plt.show()

def plot_contours_histogram(img):
    _, lengths = find_contours(img)
    plt.subplots(1,1,figsize=(10,6))
    plt.hist(lengths, bins='auto')
    plt.title("Histogram with 'auto' bins")
    plt.show()

In [20]:
def plot_boxlist_and_annotations(img_nb, boxlist, annotations) :
    fig,ax = plt.subplots(1,1,figsize=(6,6))
    
    # plot boxes
    tmp_im = np.zeros(img_collection[img_nb].shape)
    for box in boxlist:
        #print(box)
        for x in range(box[0], box[2] + box[0]) :
            for y in range(box[1], box[3] + box[1]):
                tmp_im[y,x] = 1.0
    ax.imshow(tmp_im, cmap = 'gray')
    
    # plot annotations
    if annotations != [] : 
        for anno in annotations:
            rect = patches.Rectangle((anno['bbox'][0], 
                                      anno['bbox'][1]), 
                                      anno['bbox'][2],
                                      anno['bbox'][3],
                linewidth=1,edgecolor='r',facecolor='none')
            ax.add_patch(rect)
        no_varroa = len(annotations)
    else:
        no_varroa = 0

    #ax.imshow(img_collection[img_nb], cmap='gray')
    ax.set_title('img nb {}, no varroae {}, no boxes {}'.format(
        img_nb,no_varroa, len(boxlist)))
    plt.show()

In [21]:
def plot_contours_and_annotations(img_nb, contours, annotations) :
    fig,ax = plt.subplots(1,1,figsize=(6,6))
    
    # plot contours
    for contour in contours:
        ax.plot(contour[:, 1], contour[:, 0], linewidth=2)
    
    # plot annotations
    if annotations != [] :
        for anno in annotations:
            rect = patches.Rectangle((anno['bbox'][0], 
                                      anno['bbox'][1]), 
                                      anno['bbox'][2],
                                      anno['bbox'][3],
                linewidth=1,edgecolor='r',facecolor='none')
            ax.add_patch(rect)
        no_varroa = len(annotations)
    else:
        no_varroa = 0

    #ax.imshow(img_collection[img_nb], cmap='gray')
    ax.set_title('img nb {}, no varroae {}, no contours {}'.format(
        img_nb,no_varroa, len(contours)))
    
    plt.gca().invert_yaxis()
    plt.show()